In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import keras
import neurokit2 as nk
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv1D, Dense, Reshape
from tensorflow.keras.models import Model
from tcn import TCN

from Modules.Feature_Extraction_temporal_domain import *
from Modules.tcnae import *
import warnings

warnings.filterwarnings("ignore")

In [2]:
def read_csv(filepath):
    f = open(filepath)
    data = json.load(f)
    return data

#ecg = read_csv('Preprocessed_data/ecg/ecg_unfiltered.txt')
ecg_filt = read_csv('Preprocessed_data/ecg/ecg_filtered.txt')

In [3]:
#remove first and last few minutes
for patient in ecg_filt:
    ecg_filt[patient] = ecg_filt[patient][5000:700000]
    #ecg[patient] = ecg[patient][5000:763000]

    ecg_filt[patient] = nk.ecg_clean(ecg_filt[patient], sampling_rate=62.475)

In [4]:
duration = int(10*62.4725)
segments_ecg = []

for patient in ecg_filt:
    values = ecg_filt[patient]
    for i in range(0,len(values) - duration + 1, duration):
        segments_ecg.append(values[i : (i + duration)])

In [5]:
train_df, val_df = train_test_split(segments_ecg,test_size=0.3,shuffle = True)
test_df, val_df = train_test_split(val_df,test_size=0.5,shuffle = True)

## Feature extraction from the temporal domain
Morphological/time domain features are commonly used to train the machine learning modelsWe will extract some of the common ECG features in the literature. Some features can be calculated using only R-peak locations while others requires information on the other fiducial points (P, Q, S and T waves).

#### Features from R peaks

The ___from_Rpeaks___ function calculates morphological features using R-peak locations. These features are:

'RR1': Current RR interval
'RRm': Mean of RR0, RR1 and RR2

#### Features from P, Q, R, S, T waves

The ___from_waves___ function calculates morphological features using locations of P, Q, R, S, T waves. These features are:

't_PR': Time between P and R peak locations
't_QS': Time between Q and S peak locations
't_QT':Time between Q and T peak locations
't_PT_QS': Ratio of t_PT to t_QS
't_QT_QS': Ratio of t_QT to t_QS

In [6]:
# def get_features(segments_ecg, sampling_rate, num_samples):
#
#     all_filtered_signals = []
#     all_locs_peaks = []
#     all_p_peaks_locs = []
#     all_q_peaks_locs = []
#     all_features_rpeaks = []
#     all_features_waves = []
#
#     for i in range(num_samples):
#             try:
#                 filtered_ecg, locs_peaks, p_peaks_locs, q_peaks_locs, features_rpeaks, features_waves = find_features(segments_ecg[i], sampling_rate)
#                 all_filtered_signals.append(filtered_ecg)
#                 all_locs_peaks.append(locs_peaks)
#                 all_p_peaks_locs.append(p_peaks_locs)
#                 all_q_peaks_locs.append(q_peaks_locs)
#                 all_features_rpeaks.append(features_rpeaks)
#                 all_features_waves.append(features_waves)
#             except:
#                 continue
#
#     features_waves_df = pd.DataFrame(all_features_waves)
#     features_waves_df  = features_waves_df.fillna(0)
#     features_waves_reduced = features_waves_df[["ecg_t_PR", "ecg_t_QS", "ecg_t_QT", "ecg_t_PT_QS", "ecg_t_QT_QS"]]
#
#     features_rpeaks_df = pd.DataFrame(all_features_rpeaks)
#     features_rpeaks_df  = features_rpeaks_df.fillna(0)
#
#     features_waves_reduced["ecg_RR1"] = features_rpeaks_df[["ecg_RR1"]]
#     features_waves_reduced["ecg_RRm"] = features_rpeaks_df[["ecg_RRm"]]
#
#     features_waves_reduced = features_waves_reduced.to_numpy()
#
#     return features_waves_reduced, all_filtered_signals

In [7]:
# sampling_rate = 62.475
#
# features_x_train, train_df = get_features(train_df, sampling_rate, 1000)
# features_x_val, val_df = get_features(val_df, sampling_rate, 250)
# features_x_test, test_df = get_features(test_df, sampling_rate, 250)

In [8]:
train_df = np.stack(train_df)
val_df = np.stack(val_df)
test_df = np.stack(test_df)

x_train = train_df.reshape(train_df.shape[0],train_df.shape[1], 1)
x_val = val_df.reshape(val_df.shape[0],val_df.shape[1], 1)
x_test = test_df.reshape(test_df.shape[0],test_df.shape[1], 1)

print("Training input shape: ", x_train.shape)
print("Val input shape: ", x_val.shape)
print("Test input shape: ", x_test.shape)

Training input shape:  (18124, 624, 1)
Val input shape:  (3884, 624, 1)
Test input shape:  (3884, 624, 1)


In [9]:
from Modules.utilities import *
import numpy
from tcn import TCN
import time
import tensorflow
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import UpSampling1D
from tensorflow.keras.layers import AveragePooling1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model
import pandas

class TCNAE:
    """
    A class used to represent the Temporal Convolutional Autoencoder (TCN-AE).

    ...

    Attributes
    ----------
    model : xxtypexx
        The TCN-AE model.

    Methods
    -------
    build_model(verbose = 1)
        Builds the model
    """

    model = None

    def __init__(self,
                 ts_dimension = 1,
                 dilations = (1, 2, 4, 8, 16),
                 nb_filters = 20,
                 kernel_size = 20,
                 nb_stacks = 1,
                 padding = 'same',
                 dropout_rate = 0.00,
                 filters_conv1d = 8,
                 activation_conv1d = 'linear',
                 latent_sample_rate = 42,
                 pooler = AveragePooling1D,
                 lr = 0.001,
                 conv_kernel_init = 'glorot_normal',
                 loss = 'logcosh',
                 use_early_stopping = False,
                 error_window_length = 128,
                 verbose = 1
                 ):
        """
        Parameters
        ----------
        ts_dimension : int
            The dimension of the time series (default is 1)
        dilations : tuple
            The dilation rates used in the TCN-AE model (default is (1, 2, 4, 8, 16))
        nb_filters : int
            The number of filters used in the dilated convolutional layers. All dilated conv. layers use the same number of filters (default is 20)
        """

        self.ts_dimension = ts_dimension
        self.dilations = dilations
        self.nb_filters = nb_filters
        self.kernel_size = kernel_size
        self.nb_stacks = nb_stacks
        self.padding = padding
        self.dropout_rate = dropout_rate
        self.filters_conv1d = filters_conv1d
        self.activation_conv1d = activation_conv1d
        self.latent_sample_rate = latent_sample_rate
        self.pooler = pooler
        self.lr = lr
        self.conv_kernel_init = conv_kernel_init
        self.loss = loss
        self.use_early_stopping = use_early_stopping
        self.error_window_length = error_window_length

        # build the model
        self.build_model(verbose = verbose)


    def build_model(self, verbose = 1):
        """Builds the TCN-AE model.

        If the argument `verbose` isn't passed in, the default verbosity level is used.

        Parameters
        ----------
        verbose : str, optional
            The verbosity level (default is 1)

        Returns
        -------
        KerasXYZType
        Todo

        Raises
        ------
        NotImplementedError
            If ...
        """

        tensorflow.keras.backend.clear_session()
        sampling_factor = self.latent_sample_rate
        i = Input(batch_shape=(None, None, self.ts_dimension))

        # Put signal through TCN. Output-shape: (batch,sequence length, nb_filters)
        tcn_enc = TCN(nb_filters=self.nb_filters, kernel_size=self.kernel_size, nb_stacks=self.nb_stacks, dilations=self.dilations,
                      padding=self.padding, use_skip_connections=True, dropout_rate=self.dropout_rate, return_sequences=True,
                      kernel_initializer=self.conv_kernel_init, name='tcn-enc')(i)

        # Now, adjust the number of channels...
        enc_flat = Conv1D(filters=self.filters_conv1d, kernel_size=1, activation=self.activation_conv1d, padding=self.padding)(tcn_enc)

        ## Do some average (max) pooling to get a compressed representation of the time series (e.g. a sequence of length 8)
        enc_pooled = self.pooler(pool_size=sampling_factor, strides=None, padding='valid', data_format='channels_last')(enc_flat)

        # If you want, maybe put the pooled values through a non-linear Activation
        enc_out = Activation("linear")(enc_pooled)

        # Now we should have a short sequence, which we will upsample again and then try to reconstruct the original series
        dec_upsample = UpSampling1D(size=sampling_factor)(enc_out)

        dec_reconstructed = TCN(nb_filters=self.nb_filters, kernel_size=self.kernel_size, nb_stacks=self.nb_stacks, dilations=self.dilations,
                                padding=self.padding, use_skip_connections=True, dropout_rate=self.dropout_rate, return_sequences=True,
                                kernel_initializer=self.conv_kernel_init, name='tcn-dec')(dec_upsample)

        # Put the filter-outputs through a dense layer finally, to get the reconstructed signal
        o = Dense(self.ts_dimension, activation='linear')(dec_reconstructed)

        model = Model(inputs=[i], outputs=[o])

        adam = optimizers.Adam(lr=self.lr, beta_1=0.9, beta_2=0.999, epsilon=1e-08, amsgrad=True)
        model.compile(loss=self.loss, optimizer=adam, metrics=[self.loss])
        if verbose > 1:
            model.summary()
        model.summary()
        self.model = model

    def fit(self, train_X, train_Y, batch_size=32, epochs=40, verbose = 1):
        my_callbacks = None
        if self.use_early_stopping:
            my_callbacks = [EarlyStopping(monitor='val_loss', patience=2, min_delta=1e-4, restore_best_weights=True)]

        keras_verbose = 0
        if verbose > 0:
            print("> Starting the Training...")
            keras_verbose = 2
        start = time.time()
        history = self.model.fit(train_X, train_Y,
                                 batch_size=batch_size,
                                 epochs=epochs,
                                 validation_split=0.001,
                                 shuffle=True,
                                 callbacks=my_callbacks,
                                 verbose=keras_verbose)
        if verbose > 0:
            print("> Training Time :", round(time.time() - start), "seconds.")

    def predict(self, test_X):
        X_rec =  self.model.predict(test_X)

        # do some padding in the end, since not necessarily the whole time series is reconstructed
        X_rec = numpy.pad(X_rec, ((0,0),(0, test_X.shape[1] - X_rec.shape[1] ), (0,0)), 'constant')
        E_rec = (X_rec - test_X).squeeze()
        Err = slide_window(pandas.DataFrame(E_rec), self.error_window_length, verbose = 0)
        Err = Err.reshape(-1, Err.shape[-1]*Err.shape[-2])
        sel = numpy.random.choice(range(Err.shape[0]),int(Err.shape[0]*0.98))
        mu = numpy.mean(Err[sel], axis=0)
        cov = numpy.cov(Err[sel], rowvar = False)
        sq_mahalanobis = mahalanobis_distance(X=Err[:], cov=cov, mu=mu)
        # moving average over mahalanobis distance. Only slightly smooths the signal
        anomaly_score = numpy.convolve(sq_mahalanobis, numpy.ones((50,))/50, mode='same')
        anomaly_score = numpy.sqrt(anomaly_score)
        return anomaly_score

In [10]:
#
# Build and compile the model
#
tcn_ae = TCNAE() # Use the parameters specified in the paper

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, 1)]         0         
                                                                 
 tcn-enc (TCN)               (None, None, 20)          72640     
                                                                 
 conv1d (Conv1D)             (None, None, 8)           168       
                                                                 
 average_pooling1d (Average  (None, None, 8)           0         
 Pooling1D)                                                      
                                                                 
 activation (Activation)     (None, None, 8)           0         
                                                                 
 up_sampling1d (UpSampling1  (None, None, 8)           0         
 D)                                                          

In [11]:
tcn_ae.fit(x_train, x_train, batch_size=32, epochs=10, verbose=1)


> Starting the Training...
(18124, 624, 1)
(18124, 624, 1)
Epoch 1/10


ValueError: in user code:

    File "C:\Users\Lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\Lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\Lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 1081, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "C:\Users\Lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 1139, in compute_loss
        return self.compiled_loss(
